In [1]:
import numpy as np

In [2]:
it = 0

In [22]:
it += 1
N = 50
timesteps = 100 # 10 seconds

#parameters are: zor, zoo, zoa, alpha, theta, s, sigma, tau
params = [1, 10, 10, 300, 50, 3, 5, 0.1]

init = np.random.rand(N, 4)

In [30]:
def r_ij(arr, i, j):
    diff = arr[j, :2]-arr[i, :2]
    return diff/np.linalg.norm(diff)

def ZOR(arr, i, params, boundaries, check_speed=True):
    if check_speed:
        s = params[5]
        tau = params[7]
        r_r = params[0]
        assert s*tau<r_r
    l = []
    for j in range(N):
        if j != i:
            diff = arr[j, :2]-arr[i, :2]
            if boundaries[0]<=np.linalg.norm(diff)<boundaries[1]: # make sure that fish is inside zor
                R = r_ij(arr, i, j)/np.linalg.norm(r_ij(arr, i, j))
                l.append(R)
    return -sum(l), len(l)
    
def ZOO(arr, i, params):
    r_r = params[0]
    r_o = params[1]
    l = []
    for j in range(N):
        diff = arr[j, -2:]-arr[i, -2:]
        if r_r<=np.linalg.norm(diff)<r_o: # make sure that fish is inside zoo
            R = arr[j, -2:]/np.linalg.norm(arr[j, -2:])
            l.append(R)
    return sum(l), len(l)

def ZOA(arr, i, params, boundaries, check_speed=False):
    return -ZOR(arr, i, params, boundaries)[0], ZOR(arr, i, params, boundaries)[1]
    

In [33]:
arr = init
arrays = []
for timestep in range(timesteps):
    arr_new = arr.copy()
    for i in range(N):
        zor, n_r = ZOR(arr, i, params, (0, params[0]))
        zoo, n_o = ZOO(arr, i, params)
        zoa, n_a = ZOA(arr, i, params, (params[1], params[2]))

        if n_r>0:
            vel = zor

        elif n_o>0:
            if n_a==0:
                vel = zoo
            else:
                vel = (1/2)*(zoo+zoa)

        elif n_a>0:
            if n_o==0:
                vel = zoa
            else:
                vel = (1/2)*(zoo+zoa)

        if vel.all()==0:
            vel = arr[i, -2:]

        arr_new[i] = [arr[i, 0]+arr[i, 2], arr[i, 1]+arr[i, 3], vel[0], vel[1]]
    
    arrays.append(arr_new)
    arr = arr_new
    

In [34]:
arrays

[array([[  1.36442   ,   1.41577105,  27.79678364,  -9.5602642 ],
        [  1.06483953,   0.89673843,  10.23489124,   0.32326075],
        [  1.43336053,   0.87202171,  14.88829248, -32.18173456],
        [  1.41606931,   0.71955343,  -6.21601688,   9.75915366],
        [  0.93047125,   1.15250321,  28.30430554,  11.30491004],
        [  0.61957829,   0.82595898, -12.08172015, -34.83481165],
        [  1.46811916,   0.89277587,  27.18547477,  14.4228421 ],
        [  1.06071265,   1.38872169,  36.41510247,  -0.86505174],
        [  0.83126282,   0.50288137,  -2.38302385, -23.77394021],
        [  0.91295846,   0.87703654, -27.91685195, -17.74946549],
        [  1.90619588,   1.17301085,  26.81906554,  23.55020346],
        [  0.27715558,   1.33995601, -20.72843799,  -8.8210866 ],
        [  1.12578362,   0.84096583, -28.22369897,   4.2171981 ],
        [  1.39049832,   0.71592945,   3.40093782,   4.48310921],
        [  1.42737725,   1.58401059,  14.14107325,  24.46953027],
        [ 